In [3]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
import data_loader
import config
import audio_processing
import matplotlib.pyplot as plt
from tqdm import tqdm

plt.style.use("dark_background")

In [4]:
conf = config.get_config()
f = "bright_vs_dark"
conf.update({
    "features": [f],
    "valid_split": 0.0
})

examples = data_loader.data_loader(conf)

2021-05-19 20:58:19.972 | INFO     | data_loader:data_loader:28 - Loading csv and checking audio files
2021-05-19 20:58:19.998 | INFO     | data_loader:data_loader:31 - Creating dataset


[{'bright_vs_dark': 20, 'count': 1, 'audio_file_name': 'guitar_electronic_007-060-127', 'user_id': 1}
 {'bright_vs_dark': 34, 'count': 2, 'audio_file_name': 'guitar_synthetic_001-060-025', 'user_id': 9}
 {'bright_vs_dark': 29, 'count': 1, 'audio_file_name': 'guitar_electronic_021-060-025', 'user_id': 1}
 {'bright_vs_dark': 50} {'bright_vs_dark': 9}
 {'bright_vs_dark': 89, 'count': 3, 'audio_file_name': 'guitar_electronic_046-060-127', 'user_id': 23}
 {'bright_vs_dark': 114, 'count': 3, 'audio_file_name': 'guitar_electronic_012-060-075', 'user_id': 91}
 {'bright_vs_dark': 84, 'count': 1, 'audio_file_name': 'guitar_synthetic_007-060-100', 'user_id': 1}
 {'bright_vs_dark': 82}
 {'bright_vs_dark': 50, 'count': 1, 'audio_file_name': 'guitar_electronic_013-060-025', 'user_id': 1}
 {'bright_vs_dark': 0} {'bright_vs_dark': 11}
 {'bright_vs_dark': 0, 'count': 1, 'audio_file_name': 'guitar_electronic_041-060-050', 'user_id': 1}
 {'bright_vs_dark': 0, 'count': 1, 'audio_file_name': 'guitar_acoust

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [ ]:
multiple_annots = {}

for _, value in examples.items():
    file_name = value["audio_file_name"]
    feature_val = value[f]

    if file_name not in multiple_annots:
        multiple_annots[file_name] = [feature_val]
    else:
        multiple_annots[file_name].append(feature_val)

In [ ]:
only_multiple = {}

for key, value in multiple_annots.items():
    if len(value) > 1:
        only_multiple[key] = value

In [ ]:
stats = {
    "mean": [],
    "std": [],
    "count": [],
    "count_class": []
}

for _, value in only_multiple.items():
    stats["mean"].append(np.mean(value))
    stats["std"].append(np.std(value))
    stats["count"].append(len(value))
    stats["count_class"].append(int(len(value) >= 4))

In [ ]:
df = pd.DataFrame(stats)

In [ ]:
plt.figure(figsize=(12, 8))
plt.scatter(range(len(df)), df.loc[:, "std"],
            c=df.loc[:, "count"])
plt.ylim([0., 100.])
plt.colorbar()
plt.title("standard deviation in examples with more than 1 annotations")
plt.savefig("standard_deviation.png")

In [ ]:
plt.figure(figsize=(12, 8))
plt.scatter(range(len(df)), df.loc[:, "std"],
            c=df.loc[:, "count_class"])
plt.ylim([0., 100.])
plt.colorbar()
plt.title("standard deviation, high annotations (4, 5) vs low (2)")
plt.savefig("standard_deviation_45.png")